# 1. Инициализация PySpark фреймворка

In [1]:
import numpy as np
import pandas as pd
import os

Импорт библиотек Spark SQL и Spark ML

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor

spark = SparkSession.builder.master("local[*]").getOrCreate()

Загрузка исходных данных

In [3]:
dataframe = spark.read.csv('filtered_data/filtered_data.csv', inferSchema=True, header=True, sep=';')
dataframe.limit(5).toPandas()

,timestamp,site_id,period_id,actual_consumption,actual_pv,load_00,load_19,load_38,load_57,load_76,load_95,pv_00,pv_19,pv_38,pv_57,pv_76,pv_95
0,2016-03-19 10:30:00,16,34,159.990084,1.912269,143.861074,123.932534,118.152543,134.324776,134.830698,137.857285,3.958022,146.204244,35.181755,3.695826,3.697584,9.838237
1,2016-03-19 10:45:00,16,34,119.992563,8.295386,134.507290,123.893909,121.151450,135.891911,135.030675,133.859616,28.042454,137.577075,22.077018,3.700424,3.698064,12.065509
2,2016-03-19 12:30:00,16,34,111.993059,99.026948,133.860285,121.596185,131.914748,136.694796,133.149629,132.059423,104.301363,134.664941,4.161038,3.699189,3.697935,140.617869
3,2016-03-19 14:00:00,16,34,127.992067,183.464305,123.957616,122.144601,134.170670,133.120411,141.488820,126.755303,177.470493,63.440766,4.214755,3.751745,3.703420,206.425585
4,2016-03-19 14:15:00,16,34,111.993059,218.446593,127.027236,116.827357,133.347411,136.224976,138.318585,127.472568,208.529995,67.833120,4.553095,3.787058,3.707106,214.468607


# Задача регрессии

### Подготовка данных

Выберем подмножество столбцов для использования в качестве признаков и создадим логическое поле метки с именем label со значениями 1 или 0. В качестве метки выбран столбец load_95

In [4]:
data = dataframe.select(
    "timestamp", 
    "site_id", 
    "period_id", 
    "actual_consumption", 
    "actual_pv", 
    "load_00", 
    ((col("load_95")).alias("label"))).withColumn("timestamp", unix_timestamp("timestamp").cast(DoubleType()))
data.show(10)

+-----------+-------+---------+------------------+------------------+------------------+------------------+
|  timestamp|site_id|period_id|actual_consumption|         actual_pv|           load_00|             label|
+-----------+-------+---------+------------------+------------------+------------------+------------------+
|1.4583726E9|     16|       34|159.99008431763994|1.9122687392575448|143.86107411849176|137.85728490103136|
|1.4583735E9|     16|       34|119.99256323822993| 8.295385807660148|134.50728978179805|133.85961632355335|
|1.4583798E9|     16|       34|111.99305902234796| 99.02694764955635|133.86028507820527|132.05942315683393|
|1.4583852E9|     16|       34|127.99206745411195|  183.464305472124|123.95761552132574|  126.755302914041|
|1.4583861E9|     16|       34|111.99305902234796|218.44659341392952|127.02723567316805|127.47256808926454|
|1.4583897E9|     16|       34|111.99305902234796|221.67532237053615|114.28103260681597|117.50539549192528|
|1.4583915E9|     16|       

## Разделим данные

Используем 70% данных для обучения, а 30% оставим для тестирования. В данных тестирования столбец *binary_load_00l переименован  в trueLale*l, поэтому можно использовать его позже для сравнения прогнозируемых меток с известными фактическими значениями.

In [5]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 577226  Testing Rows: 248020


Задача регрессии

In [6]:
# Создание столбца признаков для задачи регрессии

# Объединение входных столбцов в один выходной
catVect = VectorAssembler(inputCols=["timestamp", "site_id", "period_id"], outputCol="reg_catFeatures")
# Определение категориальных признаков и преобразование их в индексы
catIdx = VectorIndexer(inputCol=catVect.getOutputCol(), outputCol="reg_idxCatFeatures")
# Объединение указанных числовых столбцов в один выходной столбец
numVect = VectorAssembler(inputCols=["actual_consumption", "actual_pv", "load_00"], outputCol="reg_numFeatures")
# Масштабирование числовых признаков в диапазоне от 0 до 1
minMax = MinMaxScaler(inputCol=numVect.getOutputCol(), outputCol='reg_normFeatures')
# Объединение категориальных и числовых признаков в один общий вектор
featVect = VectorAssembler(inputCols=["reg_idxCatFeatures", "reg_numFeatures", "reg_normFeatures"], outputCol="reg_features")

# Создание модели RandomForestRegressor
rf = RandomForestRegressor(labelCol="label", featuresCol="reg_features", numTrees=10)

# Создание и выполнение пайплайна для задачи регрессии
reg_pipeline = Pipeline(stages=[catVect, catIdx, numVect, minMax, featVect, rf])
reg_model = reg_pipeline.fit(train)
reg_prediction = reg_model.transform(test)

# Вывод результатов
reg_prediction.select("reg_features", "prediction", "trueLabel").show(10, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------+------------------+------------------+
|reg_features                                                                                                                             |prediction        |trueLabel         |
+-----------------------------------------------------------------------------------------------------------------------------------------+------------------+------------------+
|[1.4374368E9,0.0,0.0,159.99008431763994,0.0,156.92691490700227,0.4557913722451246,0.0,0.5470943105951919]                                |155.5405880982158 |138.49239879171867|
|[1.4374386E9,0.0,0.0,159.99008431763994,0.0,166.4400669659378,0.4557913722451246,0.0,0.5720404056414993]                                 |162.0389551991706 |135.3021698751566 |
|[1.4374422E9,0.0,0.0,175.98909274940388,0.0,152.3725463042181,0.5013801005987444,0.0,0.5351515064292207]     

In [7]:
# Расчет метрик для задачи регрессии
reg_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = reg_evaluator.evaluate(reg_prediction)
print("RMSE = ", rmse)

RMSE =  35.69833530785684


In [8]:
# Расчет метрик для задачи регрессии
reg_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="r2")
r2 = reg_evaluator.evaluate(reg_prediction)
print("R² = ", r2)

R² =  0.7049736472975148


In [9]:
data.select(stddev('label')).show()

+-----------------+
|    stddev(label)|
+-----------------+
|65.88039398710477|
+-----------------+



CrossValidator разделяет обучающий набор данных на заданное количество складок (фолдов), обучает модель на части данных и оценивает ее на оставшихся данных. Этот процесс повторяется для каждой комбинации гиперпараметров, и выбирается модель с наилучшей средней производительностью на всех фолдах.

In [10]:
param_grid = (ParamGridBuilder()
              .addGrid(rf.maxDepth, [2, 5, 10])
              .addGrid(rf.numTrees, [5, 10, 20])
              .build())

cv = CrossValidator(estimator=reg_pipeline,
                            estimatorParamMaps=param_grid,
                            evaluator=RegressionEvaluator(),
                            numFolds=3) 

# Обучение и подбор гиперпараметров
cv_model = cv.fit(train)

best_cv_model = cv_model.bestModel

# Оценка производительности на тестовом наборе
cv_prediction = best_cv_model.transform(test)

In [11]:
reg_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = reg_evaluator.evaluate(cv_prediction)
print("RMSE = ", rmse)

RMSE =  31.186387613556807


In [12]:
# Расчет метрик для задачи регрессии
reg_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="r2")
r2 = reg_evaluator.evaluate(cv_prediction)
print("R² = ", r2)

R² =  0.7748380370545707


# Задача бинарной классификации

## Подготовка данных

Выберем подмножество столбцов для использования в качестве признаков и создадим логическое поле метки с именем label со значениями 1 или 0. В частности, *1* для показателей нагрузок более 135 в *load_95* и *0* для нагрузок менее 35.

In [13]:
data = dataframe.select(
    "timestamp", 
    "site_id", 
    "period_id", 
    "actual_consumption", 
    "actual_pv", 
    "load_00", 
    ((col("load_95") > 135).cast("Int").alias("label"))).withColumn("timestamp", unix_timestamp("timestamp").cast(DoubleType()))
data.show(10)

+-----------+-------+---------+------------------+------------------+------------------+-----+
|  timestamp|site_id|period_id|actual_consumption|         actual_pv|           load_00|label|
+-----------+-------+---------+------------------+------------------+------------------+-----+
|1.4583726E9|     16|       34|159.99008431763994|1.9122687392575448|143.86107411849176|    1|
|1.4583735E9|     16|       34|119.99256323822993| 8.295385807660148|134.50728978179805|    0|
|1.4583798E9|     16|       34|111.99305902234796| 99.02694764955635|133.86028507820527|    0|
|1.4583852E9|     16|       34|127.99206745411195|  183.464305472124|123.95761552132574|    0|
|1.4583861E9|     16|       34|111.99305902234796|218.44659341392952|127.02723567316805|    0|
|1.4583897E9|     16|       34|111.99305902234796|221.67532237053615|114.28103260681597|    0|
|1.4583915E9|     16|       34|111.99305902234796| 236.2180265473741| 129.0910128790499|    0|
|1.4583924E9|     16|       34|127.99206745411195|

## Разделим данные

Используем 70% данных для обучения, а 30% оставим для тестирования. В данных тестирования столбе ц*binary_load_00l переименован в* trueLablel, поэтому можно использовать его позже для сравнения прогнозируемых меток с известными фактическими значениями.

In [14]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 578141  Testing Rows: 247105


## Вычисление отношения между классами

In [15]:
positive_count = train.filter(col("label") == 1).count()
negative_count = train.filter(col("label") == 0).count()
balance_ratio = positive_count / negative_count
print("Positive to Negative Class Ratio:", balance_ratio)

Positive to Negative Class Ratio: 0.9859949366045021


Если balance_ratio близко к 1, это свидетельствует о балансе. Если значительно больше или меньше 1, это может указывать на дисбаланс.

In [16]:
catVect = VectorAssembler(inputCols = ["timestamp", "site_id", "period_id"], outputCol="catFeatures")
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxCatFeatures")
numVect = VectorAssembler(inputCols = ["actual_consumption", "actual_pv", "load_00"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")
featVect = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"], outputCol="features")
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
pipeline = Pipeline(stages=[catVect, catIdx, numVect, minMax, featVect, lr])

In [17]:
pipelineModel = pipeline.fit(train)

In [18]:
prediction = pipelineModel.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

+---------------------------------------------------------------------------------+----------+---------+
|features                                                                         |prediction|trueLabel|
+---------------------------------------------------------------------------------+----------+---------+
|[1.4374368E9,0.0,0.0,0.4557913722451246,0.0,0.5474120217867157]                  |1.0       |1        |
|[1.4374377E9,0.0,0.0,0.5013801005987444,0.0,0.6066605442290336]                  |1.0       |1        |
|[1.4374431E9,0.0,0.0,0.4557913722451246,0.0,0.5622126121088195]                  |1.0       |1        |
|[1.437444E9,0.0,0.0,0.3646139155378846,0.0,0.5703688811475164]                   |1.0       |0        |
|[1.4374458E9,0.0,0.0,0.3646139155378846,0.0,0.5269406893051095]                  |0.0       |1        |
|[1.4374494E9,0.0,0.0,0.4102026438915047,0.0,0.5165986830185767]                  |0.0       |1        |
|[1.4374503E9,0.0,0.0,0.4557913722451246,0.0,0.50997156

In [19]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
pr = tp / (tp + fp)
re = tp / (tp + fn)
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", pr),
 ("Recall", re),
 ("F1", 2*pr*re/(re+pr))],["metric", "value"])
metrics.show()

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|           97625.0|
|       FP|           12429.0|
|       TN|          111396.0|
|       FN|           25655.0|
|Precision|0.8870645319570393|
|   Recall|0.7918964957819598|
|       F1|0.8367833234762185|
+---------+------------------+



In [20]:
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
aur = evaluator.evaluate(prediction)
print ("AUROC = ", aur)

AUR =  0.9083527106127908


## Подбор гиперпараметров

In [21]:
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.3, 0.1])
             .addGrid(lr.maxIter, [10, 5])
             .addGrid(lr.threshold,[0.4, 0.3])
             .build())

cv = CrossValidator(estimator=pipeline, 
                    evaluator=BinaryClassificationEvaluator(), 
                    estimatorParamMaps=paramGrid, 
                    numFolds=2)

model = cv.fit(train)

In [22]:
newPrediction = model.transform(test)
newPredicted = prediction.select("features", "prediction", "trueLabel")
newPredicted.show()

+--------------------+----------+---------+
|            features|prediction|trueLabel|
+--------------------+----------+---------+
|[1.4374368E9,0.0,...|       1.0|        1|
|[1.4374377E9,0.0,...|       1.0|        1|
|[1.4374431E9,0.0,...|       1.0|        1|
|[1.437444E9,0.0,0...|       1.0|        0|
|[1.4374458E9,0.0,...|       0.0|        1|
|[1.4374494E9,0.0,...|       0.0|        1|
|[1.4374503E9,0.0,...|       1.0|        0|
|[1.4374521E9,0.0,...|       0.0|        0|
|[1.4374557E9,0.0,...|       1.0|        0|
|[1.4374566E9,0.0,...|       0.0|        0|
|[1.4374575E9,0.0,...|       1.0|        0|
|[1.4374593E9,0.0,...|       1.0|        1|
|[1.437462E9,0.0,0...|       1.0|        1|
|[1.4374692E9,0.0,...|       1.0|        1|
|[1.4374701E9,0.0,...|       1.0|        1|
|[1.437471E9,0.0,0...|       1.0|        1|
|[1.4374728E9,0.0,...|       1.0|        1|
|[1.4374746E9,0.0,...|       1.0|        1|
|[1.4374773E9,0.0,...|       1.0|        1|
|[1.4374809E9,0.0,...|       1.0

In [23]:
# Recalculate confusion matrix
tp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 1").count())
fp2 = float(newPrediction.filter("prediction == 1.0 AND truelabel == 0").count())
tn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 0").count())
fn2 = float(newPrediction.filter("prediction == 0.0 AND truelabel == 1").count())
pr2 = tp2 / (tp2 + fp2)
re2 = tp2 / (tp2 + fn2)
metrics2 = spark.createDataFrame([
 ("TP", tp2),
 ("FP", fp2),
 ("TN", tn2),
 ("FN", fn2),
 ("Precision", pr2),
 ("Recall", re2),
 ("F1", 2*pr2*re2/(re2+pr2))],["metric", "value"])
metrics2.show()

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|          116516.0|
|       FP|           54333.0|
|       TN|           69492.0|
|       FN|            6764.0|
|Precision|0.6819823352785208|
|   Recall|0.9451330304996756|
|       F1|0.7922782180607828|
+---------+------------------+



In [24]:
# Recalculate the Area Under ROC
evaluator2 = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="prediction", metricName="areaUnderROC")
aur2 = evaluator.evaluate(prediction)
print("AUR2 = ", aur2)

AUR2 =  0.9083545071236133
